# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [6]:
!cat publications.csv

pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
2023-01-01,Sustained neural representations of personally familiar people and places during cued recall,Cortex,The recall and visualization of familiar people and places is an everyday occurrence but the neural mechanisms underlying this ability are not yet well-characterized. Using magnetoencephalography (MEG) and multivariate pattern analysis we investigate the time course of familiar people and place representations in the brain. We show that distinct representations emerge ~1000 ms after cur onset and remain stable over an extended recall period (4s).,"Corriveau, A., Kidder, A., Teichmann, L., Wardle, S. G., & Baker, C. I. (2023). Sustained neural representations of personally familiar people and places during cued recall. Cortex, 158, 71–82.",SustainedRepresentations,http://annacorriveau.github.io/files/SustainedRepresentations_Cortex.pdf,
2023-04-15,Functional connectome stability and optimality are markers of co

## Import pandas

We are using the very handy pandas library for dataframes.

In [7]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [8]:
publications = pd.read_csv("publications.csv", sep=",", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2023-01-01,Sustained neural representations of personally...,Cortex,The recall and visualization of familiar peopl...,"Corriveau, A., Kidder, A., Teichmann, L., Ward...",SustainedRepresentations,http://annacorriveau.github.io/files/Sustained...,NaN
1,2023-04-15,Functional connectome stability and optimality...,Cerebral Cortex,Whole-brain patterns of fMRI functional connec...,"Corriveau, A., Yoo, K., Kwon, Y. H., Chun, M. ...",FCStability,http://annacorriveau.github.io/files/FCStabili...,NaN
2,2024-09-16,Recognition memory fluctuates with sustained a...,Psychonomic Bulletin & Review,Fluctuations in sustained attention influence ...,"Corriveau, A., Chao, A. F., deBettencourt, M. ...",RecognitionFlux,http://annacorriveau.github.io/files/Sustained...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [9]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [10]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [11]:
!ls ../_publications/

2023-01-01-SustainedRepresentations.md 2024-02-17-paper-title-number-4.md
2023-04-15-FCStability.md              2024-09-16-RecognitionFlux.md


In [12]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
